In [12]:
import gymnasium as gym
import panda_gym
from sb3_contrib import TQC
from huggingface_sb3 import load_from_hub
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
import time
import numpy as np
from panda_gym.utils import distance
import random

In [13]:
chk = load_from_hub(repo_id="BanUrsus/tqc-PandaPickAndPlace-v3", filename="tqc-PandaPickAndPlace-v3.zip")
stats = load_from_hub(repo_id="BanUrsus/tqc-PandaPickAndPlace-v3", filename="vec_normalize.pkl")
   
env = gym.make('PandaPickAndPlace-v3', autoreset=False)

sim = env.sim
robot = env.robot
task = env.task

env = DummyVecEnv([lambda: env])
env = VecNormalize.load(stats, env)
env.training = False
env.norm_reward = False

/home/frosty/Programs/Python/cse598/.venv/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.sim to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.sim` for environment variables or `env.get_wrapper_attr('sim')` that will search the reminding wrappers.
  logger.warn(
/home/frosty/Programs/Python/cse598/.venv/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.robot to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.robot` for environment variables or `env.get_wrapper_attr('robot')` that will search the reminding wrappers.
  logger.warn(
/home/frosty/Programs/Python/cse598/.venv/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.task to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.task` for

In [14]:
model = TQC.load(chk,env) 
env = model.env
obs = env.reset()

q1 = np.array([1.1 / 4 - 0.3, 0.7 / 4 , 0.4])
q2 = np.array([1.1 / 4 - 0.3, 3 * (0.7 / 4), 0.4])
q3 = np.array([(1.1 / 4) * 3 - 0.3, 0.7 / 4, 0.4])
q4 = np.array([(1.1 / 4) * 3 - 0.3, 3 * (0.7 / 4), 0.4])

random_states = []
solved_states = []

while len(random_states) < 10000 or len(solved_states) < 1000:
    action, _states = model.predict(obs, deterministic=True)
    print(action)
    obs, reward, dones, info = env.step(action)
    # gather data
    ee = robot.get_ee_position()
    fw = robot.get_fingers_width()
    block = task.get_achieved_goal()
    goal = np.array(sim.get_base_position("target"))

    # low-level state we can save that will be easy for both the planner and the robot to understand
  
    done = dones[0]
    #low-level = {
    #    "finished": done,
    #    "goal_position": np.array(sim.get_base_position("target")),
    #    "block_position": np.array(sim.get_base_position("object")),    
   #}
    
    # abstract state data
    state = {
        "effector_pos": ee,
        "finger_width": fw,

        "d_q1": distance(ee,q1),
        "d_q2": distance(ee,q2),
        "d_q3": distance(ee,q3),
        "d_q4": distance(ee,q4),
        "distance_to_block": distance(ee,block),
        "block_to_goal": distance(block, goal),
        
        "is_grasping": fw > 0.02 and fw < 0.04,
        "is_closed": fw < 0.01,
        "finished": True
    }

  
    if done:
        solved_states.append(obs)
        obs = env.reset()
    else:
        if random.randint(0, 10) == 0:
            random_states.append(obs)
print(f"finished: {len(random_states)} random states, {len(solved_states)} solved states")

[[-0.993332    0.89443815 -0.9954374   0.18217683]]
[[-0.99682707  0.73461604  0.9759033  -0.85920715]]
[[-0.9945916   0.22268331 -0.98157203  0.6548002 ]]
[[-0.9763993   0.51388836 -0.5999346  -0.93579996]]
[[ 0.99649227  0.76094556 -0.9892228  -0.86226165]]
[[ 0.9854214  -0.08474684 -0.9807249  -0.46478844]]
[[ 0.75503206  0.67507124 -0.8466531  -0.41969007]]
[[-0.34240156 -0.9707559  -0.9951732   0.12168992]]
[[-0.13750356 -0.9835348  -0.9958695   0.5019109 ]]
[[ 0.010566   -0.44840586 -0.9942342   0.6104803 ]]
[[-0.3677305   0.60691357 -0.9828261   0.46880746]]
[[ 0.89677405  0.9776504   0.83057    -0.9704005 ]]
[[ 0.99052167  0.99175334  0.9903326  -0.9919135 ]]
[[ 0.9791949   0.4456972  -0.94789547 -0.8364425 ]]
[[-0.89304316 -0.3330655  -0.9946926  -0.5038723 ]]
[[-0.8928405 -0.5816843 -0.995868  -0.4170928]]
[[-0.6960369  -0.20937371 -0.9967141   0.8210677 ]]
[[-0.70171785  0.06821954 -0.99461865  0.7441193 ]]
[[-0.4042011 -0.8998252  0.9749594 -0.8833057]]
[[-0.09393847 -0.761

KeyboardInterrupt: 

In [ ]:
# robot.get_ee_position() gets us the position of the robots "fingers"
# robot.get_fingers_width() gets us the width between the robots "fingers"

# the table has dimensions (1.1, 0.7, 0.4) w/ x_offset=-0.3
# can partition the table into 4 quadrants
# q1 = (1.1 / 4 - 0.3, 0.7 / 4 , 0.4)
# q2 = (1.1 / 4 - 0.3, 3 * (0.7 / 4), 0.4)
# q3 = ((1.1 / 4) * 3 - 0.3, 0.7 / 4, 0.4)
# q4 = ((1.1 / 4) * 3 - 0.3, 3 * (0.7 / 4), 0.4)

# distance_to_q{} = d(robot.get_ee_position(), q{})

# grasping might be distance from effectors to block & a check to see if they arent completely open or closed
# distance_to_block = d(robot.get_ee_position(), desired_goal)


In [ ]:
# task.get_achieved_goal() where the block is now
# desired_goal = np.array(sim.get_base_position("target"))
# d = distance(achieved_goal, desired_goal)

In [ ]:
# looks like overall we keep the env in the wrapped form and just save the variables
# so we can mess with them elsewhere